In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv('../train.csv')
raw_data = raw_data.drop(columns=['far_price','near_price','time_id','row_id']).dropna()
raw_data['imbalance'] = raw_data['imbalance_size']*raw_data['imbalance_buy_sell_flag']
raw_data = raw_data.drop(columns=['imbalance_size','imbalance_buy_sell_flag'])
raw_data

,stock_id,date_id,seconds_in_bucket,reference_price,matched_size,bid_price,bid_size,ask_price,ask_size,wap,target,imbalance
0,0,0,0,0.999812,13380276.64,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,3180602.69
1,1,0,0,0.999896,1642214.25,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,-166603.91
2,2,0,0,0.999561,1819368.03,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,-302879.87
3,3,0,0,1.000171,18389745.62,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,-11917682.27
4,4,0,0,0.999532,17860614.95,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,-447549.96
...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,1.000317,28280361.74,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,-2440722.89
5237976,196,480,540,1.000643,9187699.11,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,-349510.47
5237977,197,480,540,0.995789,12725436.10,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,0.00
5237978,198,480,540,0.999210,94773271.05,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,1000898.84


In [3]:
train_val_data = raw_data.query('date_id < 478')
train_data = train_val_data[:-2700]
val_data = train_val_data[-2700:]
test_data = raw_data.query('date_id >= 478')

In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 스케일링을 적용할 특정 컬럼 선택
column_to_scale = ['matched_size','bid_size','ask_size','imbalance']
for col in column_to_scale:
  column_values = train_data[col].values.reshape(-1, 1)
  column_val_values = val_data[col].values.reshape(-1, 1)
  column_test_values = test_data[col].values.reshape(-1, 1)

  # Min-Max 스케일러 생성 및 적용
  scaler = MinMaxScaler()
  scaled_values = scaler.fit_transform(column_values)
  scaled_test_values = scaler.transform(column_test_values)

  # 결과를 데이터프레임에 반영
  train_data[col] = scaled_values
  test_data[col] = scaled_test_values

# 결과 확인
print(train_data)


         stock_id  date_id  seconds_in_bucket  reference_price  matched_size  \
0               0        0                  0         0.999812      0.001734   
1               1        0                  0         0.999896      0.000212   
2               2        0                  0         0.999561      0.000235   
3               3        0                  0         1.000171      0.002383   
4               4        0                  0         0.999532      0.002314   
...           ...      ...                ...              ...           ...   
5204975       195      477                540         1.002359      0.004575   
5204976       196      477                540         0.998663      0.001142   
5204977       197      477                540         0.997623      0.001343   
5204978       198      477                540         0.999832      0.008262   
5204979       199      477                540         0.996704      0.002943   

         bid_price  bid_size  ask_price

C:\Users\John\AppData\Local\Temp\ipykernel_18952\322240145.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[col] = scaled_values
C:\Users\John\AppData\Local\Temp\ipykernel_18952\322240145.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[col] = scaled_test_values
C:\Users\John\AppData\Local\Temp\ipykernel_18952\322240145.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [41]:
def create_sequences(train_data):
    xs = []
    ys = []
    
    for date in range(0,478):
        print(f'date_{date}')
        for second in range(0, 541, 10):
            data = train_data.query(f'date_id=={date} and seconds_in_bucket == {second}')

            if len(data) == 200:
                x = data.drop(columns=['target','date_id','seconds_in_bucket', 'stock_id'])
                y = data['target']

                xs.append(x)
                ys.append(y)
            else:
                for id in range(0,200):
                    if id in data.stock_id.values:
                        continue
                    else:
                        new_row = pd.DataFrame([[id,date,second,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]],columns = data.columns)
                        data = pd.concat([data,new_row])

                data = data.sort_values('stock_id').reset_index(drop=True)

                x = data.drop(columns=['target','date_id','seconds_in_bucket', 'stock_id'])
                y = data['target']

                xs.append(x)
                ys.append(y)

    return np.array(xs), np.array(ys)

def create_test_sequences(test_data):
    xs = []
    ys = []

    for date in range(478,481):
        print(f'date_{date}')
        for second in range(0, 541, 10):
            data = test_data.query(f'date_id=={date} and seconds_in_bucket == {second}')

            if len(data) == 200:
                x = data.drop(columns=['target','date_id','seconds_in_bucket', 'stock_id'])
                y = data['target']

                xs.append(x)
                ys.append(y)
            else:
                for id in range(0,200):
                    if id in data.stock_id.values:
                        continue
                    else:
                        new_row = pd.DataFrame([[id,date,second,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]],columns = data.columns)
                        data = pd.concat([data,new_row])

                data = data.sort_values('stock_id').reset_index(drop=True)

                x = data.drop(columns=['target','date_id','seconds_in_bucket', 'stock_id'])
                y = data['target']

                xs.append(x)
                ys.append(y)

    return np.array(xs), np.array(ys)

In [42]:
train_data

,stock_id,date_id,seconds_in_bucket,reference_price,matched_size,bid_price,bid_size,ask_price,ask_size,wap,target,imbalance
0,0,0,0,0.999812,0.001734,0.999812,0.002002,1.000026,0.000156,1.000000,-3.029704,0.708960
1,1,0,0,0.999896,0.000212,0.999896,0.000107,1.000660,0.000379,1.000000,-5.519986,0.708165
2,2,0,0,0.999561,0.000235,0.999403,0.001253,1.000298,0.000349,1.000000,-8.389950,0.708132
3,3,0,0,1.000171,0.002383,0.999999,0.000077,1.000214,0.008805,1.000000,-4.010200,0.705374
4,4,0,0,0.999532,0.002314,0.999394,0.000544,1.000016,0.000008,1.000000,-7.349849,0.708098
...,...,...,...,...,...,...,...,...,...,...,...,...
5204975,195,477,540,1.002359,0.004575,1.002359,0.014030,1.002482,0.005999,1.002429,-3.190041,0.707537
5204976,196,477,540,0.998663,0.001142,0.998663,0.001198,0.999168,0.001747,0.998803,-6.200075,0.707911
5204977,197,477,540,0.997623,0.001343,0.997240,0.000688,0.997623,0.000506,0.997405,0.000000,0.708325
5204978,198,477,540,0.999832,0.008262,0.999712,0.017775,0.999952,0.017787,0.999797,1.300573,0.706845


In [43]:
#학습용, 검증용, 시험용으로 분리
X_train, y_train = create_sequences(train_data)

date_0
date_1
date_2
date_3
date_4
date_5
date_6
date_7
date_8
date_9
date_10
date_11
date_12
date_13
date_14
date_15
date_16
date_17
date_18
date_19
date_20
date_21
date_22
date_23
date_24
date_25
date_26
date_27
date_28
date_29
date_30
date_31
date_32
date_33
date_34
date_35
date_36
date_37
date_38
date_39
date_40
date_41
date_42
date_43
date_44
date_45
date_46
date_47
date_48
date_49
date_50
date_51
date_52
date_53
date_54
date_55
date_56
date_57
date_58
date_59
date_60
date_61
date_62
date_63
date_64
date_65
date_66
date_67
date_68
date_69
date_70
date_71
date_72
date_73
date_74
date_75
date_76
date_77
date_78
date_79
date_80
date_81
date_82
date_83
date_84
date_85
date_86
date_87
date_88
date_89
date_90
date_91
date_92
date_93
date_94
date_95
date_96
date_97
date_98
date_99
date_100
date_101
date_102
date_103
date_104
date_105
date_106
date_107
date_108
date_109
date_110
date_111
date_112
date_113
date_114
date_115
date_116
date_117
date_118
date_119
date_120
date_121
date_122
dat

In [45]:
X_train.shape

(26290, 200, 8)

In [44]:
X_test, y_test = create_test_sequences(test_data)

date_478
date_479
date_480


In [46]:
X_test.shape

(165, 200, 8)

In [ ]:
X_train, y_train = X_train[:-2700], y_train[:-2700]
X_val, y_val = X_train[-2700:], y_train[-2700:]

In [ ]:
dataset_optiver_ANN = {'X_train':X_train,'X_val': X_val,'X_test': X_test,'y_train': y_train,'y_val': y_val,'y_test': y_test}

In [ ]:
import pickle
# save
with open('dataset_optiver_ANN.pickle', 'wb') as f:
    pickle.dump(dataset_optiver_ANN, f, pickle.HIGHEST_PROTOCOL)